In [4]:
import requests
import json
from bs4 import BeautifulSoup
import re
import psycopg2
import time
import numpy as np

# Data architecture

###Table Artist:
artist_url - string
artist_name - string

###Table Producer:
producer_url - string
producer_name - string

###Table Song:
song_url - string
song_name - string
album_name - string
label - string
year - int
main_genre - string

###Table ArtistToSong:
artist_url - string
song_url - string

###Table ProducerToSong:
producer_url - string
song_url - string

###Table SongRelationship:
song1 - string
song2 - string
relationship - int (1 for sampled from, 2 for sampled by, 3 for cover of, 4 for covered by, 5 for remix of, and 6 for remixed by)

In [16]:
class GetData(object):
    def __init__(self):
        self.url_base = 'http://www.whosampled.com'
        self.used_genres = set([])
        self.unused_genres = set(['/genre/Hip-Hop/', '/genre/Electronic-Dance/', '/genre/Rock-Pop/', \
                                '/genre/Soul-Funk-Disco/', '/genre/Jazz-Blues/', '/genre/Reggae/', \
                                '/genre/Country-Folk/', '/genre/World/', '/genre/Soundtrack/', '/genre/Classical/'])
        self.used_artists = set([])
        self.unused_artists = set([])
        self.used_producers = set([])
        self.used_songs = set([])
        self.unused_songs = set([])

    # Create soup object
    def _get_soup(self, url):
        '''
        INPUT: url string
        OUTPUT: soup object for processing
        '''
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'html.parser')
        return soup

    # Collect all artist names based on genre
    def _get_artists_by_genre(self, genre):
        '''
        INPUT: genre string
        OUTPUT: saves artists names into SQL database
        '''
        artists = []
        url = self.url_base + genre
        genre_soup = self._get_soup(url)
        alpha_urls = [x['href'] for x in genre_soup.select('.browse-letters a')]
        for alphabet_url in alpha_urls:
            pages = int(self._get_num_pages(alphabet_url))
            decomp_url = alphabet_url.split('/')
            for page in xrange(pages):
                decomp_url[-2] = str(page + 1)
                page_url = '/'.join(decomp_url)
                print page_url
                artists += self._get_link_and_text_data(page_url, '.artistName a')
        
        conn = None
        conn = psycopg2.connect(database = 'beatlineage', user = 'gSchool', host = '/tmp')
        cur = conn.cursor()
        
        for artist in artists:
            if artist[0] in self.used_artists:
                continue
            else:
                self.unused_artists.add(artist[0])
                cur.execute('INSERT INTO artists VALUES(%s, %s)', artist)
                conn.commit()
                
        if conn:
            cur.close()
            conn.close()
    
    # Collect all song names based on artist
    def _get_songs_by_artist(self, artist):
        '''
        INPUT: artist string
        OUTPUT: saves songs into list
        '''
        songs = []
        pages = int(self._get_num_pages(artist))
        for pg_num in xrange(pages):
            page_url = artist + '?sp=' + str(pg_num + 1)
            songs += self._get_link_and_text_data(page_url, '.trackName a[itemprop="url"]')
        
        for song in songs:
            if song[0] in self.used_songs:
                continue
            else:
                self.unused_songs.add(song[0])

    # Collect all data for a single song
    def _get_song_data(self, song):
        '''
        INPUT: song string
        OUTPUT: saves producer and song data into SQL database
        '''
        producers = []
        self.sampled_from_song_url = []
        self.sampled_by_song_url = []
        self.cover_of_song_url = []
        self.covered_by_song_url = []
        self.remix_of_song_url = []
        self.remixed_by_song_url = []
        url = self.url_base + song
        soup = self._get_soup(url)
        
        # Basic song info
        try:
            name = soup.select('h1[itemprop="name"]')[0].text
        except:
            name = ''
        artists = [x['href'] for x in soup.select('h2 a')]
        try:
            album = soup.select('.trackReleaseDetails h3')[0].text
        except:
            album = ''
        try:
            main_label = re.split('; |, |\*|\n', soup.select('.trackReleaseDetails h4')[0].text)[0]
        except:
            main_label = ''
        try:
            year = int(soup.select('.trackReleaseDetails')[0].contents[4].strip())
        except:
            year = 0
        if len(soup.select('.trackReleaseDetails a')) > 0:
            producers = self._get_link_and_text_data(song, '.trackReleaseDetails a')
        else:
            producers = ''  
        
        # Song sampling history
        for header in soup.select('.sectionHeader'):
            if header.select('.moreButton'):
                more_url = header.select('.moreButton')[0]['href']
                pages = int(self._get_num_pages(more_url))
                for pg_num in xrange(pages):
                    page_url = more_url + '?cp=' + str(pg_num + 1)
                    page_soup = self._get_soup(self.url_base + page_url)
                    for header2 in page_soup.select('.sectionHeader'):
                        self._get_sample_data(header2)
            else:
                self._get_sample_data(header)
        
        # Open a connection to postgresql
        conn = None
        conn = psycopg2.connect(database = 'beatlineage', user = 'gSchool', host = '/tmp')
        cur = conn.cursor()
        
        # Write to SQL tables
        # Saves producer info and relationship mapping
        for producer in producers:
            if producer[0] in self.used_producers:
                continue
            else:
                self.used_producers.add(producer[0])
                cur.execute('INSERT INTO producers VALUES(%s, %s)', producer)
                conn.commit()
            cur.execute('INSERT INTO producer_to_song VALUES(%s, %s)', (producer[0], song))
            conn.commit()
        
        # Saves song info
        cur.execute('INSERT INTO songs VALUES(%s, %s, %s, %s, %s, %s)', (song, name, album, main_label, year, self.song_genre))
        conn.commit()
        
        # Saves artist to song relationship mapping
        for artist in artists:
            cur.execute('INSERT INTO artist_to_song VALUES(%s, %s)', (artist, song))
            conn.commit()
        
        # Saves song to song relationship mapping
        tables = [self.sampled_from_song_url, self.sampled_by_song_url, self.cover_of_song_url, self.covered_by_song_url, self.remix_of_song_url, self.remixed_by_song_url]
        for i, table in enumerate(tables):
            self._write_song_to_song_table(cur, conn, song, table, i+1)
        
        # Close connection to postgresql
        if conn:
            cur.close()
            conn.close()

    # Get the sampling data
    def _get_sample_data(self, header):
        '''
        INPUT: song string
        OUTPUT: no returns, just update self variables
        '''
        ul = header.find_next_sibling('ul')
        if header.text.find('samples of') != -1:
            sampled_from_song_urls = self._get_genre_and_song_url(ul, 0)
            self.sampled_from_song_url += sampled_from_song_urls
        elif header.text.find('sampled in') != -1:
            sampled_by_song_urls = self._get_genre_and_song_url(ul, 1)
            self.sampled_by_song_url += sampled_by_song_urls
        elif header.text.find('cover of') != -1:
            cover_of_song_urls = self._get_genre_and_song_url(ul, 0)
            self.cover_of_song_url += cover_of_song_urls
        elif header.text.find('covered in') != -1:
            covered_by_song_urls = self._get_genre_and_song_url(ul, 1)
            self.covered_by_song_url += covered_by_song_urls
        elif header.text.find('remix of') != -1:
            remix_of_song_urls = self._get_genre_and_song_url(ul, 0)
            self.remix_of_song_url += remix_of_song_urls
        elif header.text.find('remixed in') != -1:
            remixed_by_song_urls = self._get_genre_and_song_url(ul, 1)
            self.remixed_by_song_url += remixed_by_song_urls
        else:
            pass

    # Uses the urls from _get_sample_data to grab the original song's genre and the new song's url
    # Side is binary for the side of the original song, 0 for left (sample of, cover of, remix of) and
    # 1 for right (sampled in, covered in, remixed in)
    def _get_genre_and_song_url(self, ul, side):
        '''
        INPUT: unordered list object and the side of the original song
        OUTPUT: original song genre and list of new song urls
        '''
        url_list = [x['href'] for x in ul.select('.trackName')]
        song_urls = []
        for url in url_list:
            pg_url = self.url_base + url
            soup = self._get_soup(pg_url)
            genres = [x['href'] for x in soup.select('.sampleAdditionalInfoContainer a[href^="/genre"]')]
            if len(genres) == 0:
                new_song_genre = '/browse/artists/Other/A/1/'
            elif len(genres) == 1 and self.song_genre != '/browse/artists/Other/A/1/':
                new_song_genre = '/browse/artists/Other/A/1/'
            elif len(genres) == 1 and self.song_genre == '/browse/artists/Other/A/1/':
                new_song_genre = genres[0]
            else:
                new_song_genre = genres[not side]
            
            song_urls.append([x['href'] for x in soup.select('h3 .trackName')][not side])
            
            if new_song_genre in self.used_genres:
                continue
            else:
                self.unused_genres.add(new_song_genre)
        return song_urls
    
    # Get the number of pages on a page
    def _get_num_pages(self, end_url):
        '''
        INPUT: ending url string
        OUTPUT: number of pages of request
        '''
        url = self.url_base + end_url
        page_soup = self._get_soup(url)
        try:
            num_of_pg = int(page_soup.select('.page')[-1].text)
        except:
            num_of_pg = 1
        return num_of_pg
    
    # Extract text and link data for a given CSS tag
    def _get_link_and_text_data(self, end_url, tag):
        '''
        INPUT: ending url and CSS tag
        OUTPUT: list of tuple of (text, link)
        '''
        url = self.url_base + end_url
        page_soup = self._get_soup(url)
        text = [x.text for x in page_soup.select(tag)]
        link = [x['href'] for x in page_soup.select(tag)]
        return zip(link, text)
    
    # Write to song2song SQL table
    def _write_song_to_song_table(self, cur, conn, song1, song_list, code):
        for song in song_list:
            cur.execute('INSERT INTO song_to_song VALUES(%s, %s, %s)', (song1, song, code))
            conn.commit()
    
    # Used to scrape for all songs
    def get_songs(self):
        while len(self.unused_genres) > 0:
            genre = self.unused_genres.pop()
            self.used_genres.add(genre)
            self._get_artists_by_genre(genre)
            self.song_genre = genre
            while len(self.unused_artists) > 0:
                artist = self.unused_artists.pop()
                print artist
                self.used_artists.add(artist)
                self._get_songs_by_artist(artist)
                while len(self.unused_songs) > 0:
                    song = self.unused_songs.pop()
                    self.used_songs.add(song)
                    self._get_song_data(song)
                time.sleep(2 * np.random.rand())

In [17]:
data = GetData()

In [7]:
conn = None
conn = psycopg2.connect(database = 'beatlineage', user = 'gSchool', host = '/tmp')
cur = conn.cursor()
conn.autocommit = True

In [220]:
# Create artist table in SQL
def create_artist_table(cur, con):
    try:
        cur.execute('CREATE TABLE artists (artist_url varchar, \
                                           artist_name varchar)')
    except:
        pass

    # Example entry:
    #cur.execute('INSERT INTO artists VALUES(%s, %s)', ('/Kanye-West/', 'Kanye West'))

create_artist_table(cur, conn)

In [221]:
# Create song table in SQL
def create_song_table(cur, con):
    try:
        cur.execute('CREATE TABLE songs (song_url varchar, \
                                         song_name varchar, \
                                         album_name varchar, \
                                         label_name varchar, \
                                         year integer, \
                                         main_genre varchar)')
    except:
        pass

    # Example entry:
    #cur.execute('INSERT INTO songs VALUES(%s, %s, %s, %s, %s, %s)', ('/Lil-Jon/Lovers-%26-Friends/', 'Lovers & Friends', 'Crunk Juice', 'TVT', 2004, '/genre/Hip-Hop/'))

create_song_table(cur, conn)

In [222]:
# Create producer table in SQL
def create_producer_table(cur, con):
    try:
        cur.execute('CREATE TABLE producers (producer_url varchar, \
                                             producer_name varchar)')
    except:
        pass

    # Exmaple entry:
    #cur.execute('INSERT INTO producers VALUES(%s, %s)', ('/Kanye-West/', 'Kanye West'))

create_producer_table(cur, conn)

In [223]:
def create_artist_to_song_table(cur, con):
    try:
        cur.execute('CREATE TABLE artist_to_song (artist_url varchar, \
                                                  song_url varchar)')
    except:
        pass

    # Example entry:
    #cur.execute('INSERT INTO artist_to_song VALUES(%s, %s)', ('/Kanye-West/', '/Kanye-West/Mercy/'))

create_artist_to_song_table(cur, conn)

In [224]:
# Create producer to song mapping table in SQL
def create_producer_to_song_table(cur, con):
    try:
        cur.execute('CREATE TABLE producer_to_song (producer_url varchar, \
                                                    song_url varchar)')
    except:
        pass

    # Exmaple entry:
    #cur.execute('INSERT INTO producer_to_song VALUES(%s, %s)', ('/Kanye-West/', '/Kanye-West/Mercy/Power'))

create_producer_to_song_table(cur, conn)

In [225]:
# Create song to song relatioship table in SQL
def create_song_to_song_table(cur, con):
    try:
        cur.execute('CREATE TABLE song_to_song (song1_url varchar, \
                                                song2_url varchar, \
                                                relationship integer)')
    except:
        pass

    # Example entry:
    #cur.execute('INSERT INTO song_to_song VALUES(%s, %s, %s)', ('/Black-Eyed-Peas/Boom-Boom-Pow/', '/Daft-Punk/Harder,-Better,-Faster,-Stronger/', 1))

create_song_to_song_table(cur, conn)

In [80]:
data._get_songs()

/browse/artists/Soul-Funk-Disco/W/1/
/browse/artists/Soul-Funk-Disco/W/2/
/browse/artists/Soul-Funk-Disco/W/3/
/browse/artists/Soul-Funk-Disco/W/4/
/browse/artists/Soul-Funk-Disco/W/5/
/browse/artists/Soul-Funk-Disco/W/6/
/browse/artists/Soul-Funk-Disco/W/7/
/browse/artists/Soul-Funk-Disco/W/8/
/browse/artists/Soul-Funk-Disco/W/9/
/browse/artists/Soul-Funk-Disco/W/10/
/browse/artists/Soul-Funk-Disco/W/11/
/browse/artists/Soul-Funk-Disco/W/12/
/browse/artists/Soul-Funk-Disco/W/13/
/William-Weatherspoon/?sp=1
/Wing-and-a-Prayer-Fife-and-Drum-Corps/?sp=1
/Warren-Moore/?sp=1
/We-Productions/?sp=1
/William-Stevenson/?sp=1
/William-Stevenson/?sp=2
/Willy-Albimoor/?sp=1
/William-%22Mickey%22-Stevenson/?sp=1
/Willie-Henderson/?sp=1
/Warren-Schatz/?sp=1
/Wilkins/?sp=1
/Willie-Tee/?sp=1
/Wynder-K.-Frog/?sp=1
/Walter-Murphy/?sp=1
/Walter-Murphy/?sp=2
/Wild-Sugar/?sp=1
/William-Salter/?sp=1
/Willie-Clarke/?sp=1
/Willie-Clarke/?sp=2
/Winston-Johnson/?sp=1
/Waylon/?sp=1
/Whiskey-Barons/?sp=1
/White-

In [146]:
url = 'http://www.whosampled.com/sample/64343/Black-Eyed-Peas-Boom-Boom-Pow-Daft-Punk-Harder,-Better,-Faster,-Stronger/'
html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')
print [x['href'] for x in soup.select('.sampleAdditionalInfoContainer a[href^="/genre"]')]
print [x['href'] for x in soup.select('h3 .trackName')]
#[x['href'] for x in soup.select('.sampleAdditionalInfoContainer')]

[u'/genre/Hip-Hop/', u'/genre/Electronic-Dance/']
[u'/Black-Eyed-Peas/Boom-Boom-Pow/', u'/Daft-Punk/Harder,-Better,-Faster,-Stronger/']


In [263]:
data.unused_genres

{'/genre/Classical/',
 '/genre/Country-Folk/',
 '/genre/Electronic-Dance/',
 '/genre/Hip-Hop/',
 '/genre/Jazz-Blues/',
 '/genre/Reggae/',
 '/genre/Rock-Pop/',
 '/genre/Soul-Funk-Disco/',
 '/genre/Soundtrack/',
 '/genre/World/'}

In [18]:
data.get_songs()

/browse/artists/Classical/A/1/
/browse/artists/Classical/A/2/
/browse/artists/Classical/A/3/
/browse/artists/Classical/A/4/
/browse/artists/Classical/A/5/
/browse/artists/Classical/A/6/
/browse/artists/Classical/A/7/
/browse/artists/Classical/B/1/
/browse/artists/Classical/B/2/
/browse/artists/Classical/B/3/
/browse/artists/Classical/C/1/
/browse/artists/Classical/C/2/
/browse/artists/Classical/C/3/
/browse/artists/Classical/C/4/
/browse/artists/Classical/C/5/
/browse/artists/Classical/D/1/
/browse/artists/Classical/D/2/
/browse/artists/Classical/D/3/
/browse/artists/Classical/E/1/
/browse/artists/Classical/E/2/
/browse/artists/Classical/E/3/
/browse/artists/Classical/E/4/
/browse/artists/Classical/F/1/
/browse/artists/Classical/F/2/
/browse/artists/Classical/F/3/
/browse/artists/Classical/G/1/
/browse/artists/Classical/G/2/
/browse/artists/Classical/G/3/
/browse/artists/Classical/G/4/
/browse/artists/Classical/G/5/
/browse/artists/Classical/H/1/
/browse/artists/Classical/H/2/
/browse/

ReadTimeout: HTTPConnectionPool(host='www.whosampled.com', port=80): Read timed out. (read timeout=None)